In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.action_chains import ActionChains
import requests as rq
from bs4 import BeautifulSoup as bs 
import numpy as np
import time
import gzip
import shutil
import xmltodict
import os
import matplotlib.pyplot as plt
import math
import pandas as pd

In [ ]:
driver=wd.Chrome()
def click(xpath):
    driver.find_element('xpath',xpath).click()
url='https://roadsafety.tw/AccLocCbi'
driver.maximize_window()
driver.get(url)
time.sleep(1)
click('//*[@id="DocModal"]/div/div/div[1]/button')
click('//*[@id="mdAnnounce"]/div/div/div[1]/button/span')
def text(xpath):
    return driver.find_element('xpath',xpath).text

In [ ]:
for i in range(2):
    click(f'//*[@id="ddlCyear"]/option[{2+i}]')
    print(text(f'//*[@id="ddlCyear"]/option[{2+i}]'))
    click('//*[@id="ddlCity"]/option[13]')
    click('//*[@id="ddlCalculation"]/option[2]')
    for j in range(2,19+1):
        click(f'//*[@id="ddlArea"]/option[{j}]')
        name=text(f'//*[@id="ddlArea"]/option[{j}]')
        print(name)
        click('//*[@id="bSearch"]') 
        time.sleep(0.1)
        table10=text('//*[@id="tbMain"]')
        for line in table10.split('\n'):
                if line[0].isdigit():
                    print(line)
        time.sleep(1)
        for k in range(len(table10.split('\n'))-6):
            print(table10.split('\n')[k+6].split()[2])
            click(f'//*[@id="tbMain"]/tbody/tr[{k+1}]')
            time.sleep(1)
            try:
                click('//*[@id="pQtyInfo"]/ul/li[3]/a')
                time.sleep(0.1)
                table=text('//*[@id="tbHistory"]')
            except:
                click('//*[@id="bAdditional"]')
                click('//*[@id="pQtyInfo"]/ul/li[3]/a')
                time.sleep(0.1)
                table=text('//*[@id="tbHistory"]')
            print('年度 件數 死亡人數 受傷人數')
            for line in table.split('\n'):
                if line[0].isdigit():
                    print(line)
        # print(table)

In [ ]:
intersections={}
for i in range(2):
    click(f'//*[@id="ddlCyear"]/option[{2+i}]')
    year=text(f'//*[@id="ddlCyear"]/option[{2+i}]')
    print(year)
    click('//*[@id="ddlCity"]/option[13]')
    click('//*[@id="ddlCalculation"]/option[2]')
    for j in range(2,19+1):
        click(f'//*[@id="ddlArea"]/option[{j}]')
        name=text(f'//*[@id="ddlArea"]/option[{j}]')
        print(name)
        click('//*[@id="bSearch"]') 
        time.sleep(0.1)
        table10=text('//*[@id="tbMain"]')
        time.sleep(1)
        for k in range(len(table10.split('\n'))-6):
            intersection=name+table10.split('\n')[k+6].split()[2]
            click(f'//*[@id="tbMain"]/tbody/tr[{k+1}]')
            time.sleep(0.8)
            try:
                click('//*[@id="pQtyInfo"]/ul/li[3]/a')
                time.sleep(0.5)
                table=text('//*[@id="tbHistory"]')
            except:
                click('//*[@id="bAdditional"]')
                time.sleep(0.5)
                click('//*[@id="pQtyInfo"]/ul/li[3]/a')
                time.sleep(0.5)
                table=text('//*[@id="tbHistory"]')
            if intersection not in intersections:
                intersections[intersection]=table
            else:
                if table==intersections[intersection]:
                    print('same')
                else:
                    intersections[f'{intersection}_{year}_{k+1}']=table

In [ ]:
intersections_array={}
for key in intersections:
    a=[]
    for line in intersections[key].split('\n')[1:]:
        lst=line.split()
        if lst[0]=='111' or lst[0]=='112':
            a.append(lst)
    if 111 not in np.array(a,dtype=int)[:,0]:
        a.append([111,0,0,0])
    if 112 not in np.array(a,dtype=int)[:,0]:
        a.append([112,0,0,0])
    for i in range(len(a)):
        for j in range(len(a[i])):
            a[i][j]=int(a[i][j])
    a.sort(key=lambda x:x[0])
    a=np.array(a,dtype=int)
    a=a[:,1:]
    a=np.vstack((a,a.sum(axis=0)))
    intersections_array[key]=a
#rank the intersections in intersections_array by the number of accidents, i.e. the last column of the last row
intersections_array_ranked=sorted(intersections_array.items(),key=lambda x:x[1][-1][-1],reverse=True)
for i in intersections_array_ranked:
    print(i[0],end=' ')
    for j in range(len(i[1])):
        for k in range(len(i[1][j])):
            print(i[1][j][k],end=' ')
    print()